In [1]:
import cv2
import numpy as np
import mediapipe as mp

In [2]:
Drawing = mp.solutions.drawing_utils
Drawing_Spec = Drawing.DrawingSpec(thickness=2, circle_radius=4, color=(0,255,127)) 

MP_Pose = mp.solutions.pose
Pose = MP_Pose.Pose(
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7
    )

## Angle Calculation

In [3]:
def Calculate_Angle(Point_1, Point_2, Point_3):
    Point_1 = np.array(Point_1) 
    Point_2 = np.array(Point_2)
    Point_3 = np.array(Point_3) 
    
    radians = np.arctan2(Point_3[1]-Point_2[1], Point_3[0]-Point_2[0]) - np.arctan2(Point_1[1]-Point_2[1], Point_1[0]-Point_2[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

![Body Landmarks ](Assets//Body_LandMarks.png)

## Main Angles Definition

In [9]:
Main_Angles = {
    
    "Neck"       : [0,11,12]  , 
    "L_Shoulder" : [13,11,23] , 
    "R_Shoulder" : [14,12,24] , 
    "L_Elbow"    : [11,13,15] , 
    "R_Elbow"    : [12,14,16] ,
    "L_Wrist"    : [13,15,19] , 
    "R_Wrist"    : [14,16,20] ,
    "L_Hip"      : [11,23,25] , 
    "R_Hip"      : [12,24,26] ,
    "L_Ankle"    : [25,27,31] ,
    "R_Ankle"    : [26,28,32] , 
}

## Draw Main Angles 

In [18]:
def Draw_Angles( image , L_Marks ) :

    for Key, Val in Main_Angles.items():

        Angle = int(Calculate_Angle([L_Marks[Val[0]].x, L_Marks[Val[0]].y],
                                [L_Marks[Val[1]].x, L_Marks[Val[1]].y],
                                [L_Marks[Val[2]].x, L_Marks[Val[2]].y]))
        txt = tuple(np.multiply([L_Marks[Val[1]].x, L_Marks[Val[1]].y],[640,480]).astype(int))

        # Text settings
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.7
        font_thickness = 2
        text_color = (209, 196, 233) 

        text_size, _ = cv2.getTextSize(str(Angle), cv2.FONT_HERSHEY_SIMPLEX, font_scale, font_thickness)
        cv2.rectangle(image, txt, (txt[0] + text_size[0], txt[1] - text_size[1]),(0, 0, 102), cv2.FILLED)

        cv2.putText(image, str(Angle), txt, font, font_scale, text_color, font_thickness, cv2.LINE_AA)
    return image

## Pose Estimation

In [19]:
cap = cv2.VideoCapture(0)

while cap.isOpened():

    ret, frame = cap.read()

    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the image using the mediapipe pose model
    results = Pose.process(image)

    # Check if pose landmarks are present
    if results.pose_landmarks:
        L_Marks = results.pose_landmarks.landmark

        Drawing.draw_landmarks(
            image,
            results.pose_landmarks,
            MP_Pose.POSE_CONNECTIONS,
            landmark_drawing_spec=Drawing_Spec,
            connection_drawing_spec=Drawing_Spec
        )
        image = Draw_Angles(image, L_Marks)



    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    cv2.imshow('Phlex', image)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
